# Train a proprioception-tuned CNN

We create a sensor processing model using CNN-based visual encoding finetuned with proprioception.

We create an encoding for the robot starting from a pretrained CNN model. As the feature vector of this is still large (eg 512 * 7 * 7), we reduce this to the encoding with an MLP. 

We finetune the encoding with information from proprioception.  

The sensor processing object associated with the network trained like this is in sensorprocessing/sp_propriotuned_cnn.py

In [1]:
import sys
sys.path.append("..")

from exp_run_config import Config, Experiment
Config.PROJECTNAME = "BerryPicker"

import pathlib
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from demonstration.demonstration import Demonstration

import sensorprocessing.sp_helper as sp_helper
from sensorprocessing.sp_propriotuned_cnn import VGG19ProprioTunedRegression, ResNetProprioTunedRegression
from robot.al5d_position_controller import RobotPosition

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [12]:
# 
external_path = pathlib.Path("/Users/lboloni/Documents/Code/_ExternalExp")
external_path.exists()
Config().set_experiment_path(external_path)
Config().copy_experiment("sensorprocessing_propriotuned_cnn")
Config().copy_experiment("robot_al5d")
Config().copy_experiment("demonstration")

Config().create_exprun_variant("sensorprocessing_propriotuned_cnn","resnet50_128", {"epochs": 17}, new_run_name="boo")

***ExpRun**: Experiment config path changed to \Users\lboloni\Documents\Code\_ExternalExp
***ExpRun**: Experiment sensorprocessing_propriotuned_cnn copied to \Users\lboloni\Documents\Code\_ExternalExp\sensorprocessing_propriotuned_cnn
***ExpRun**: Experiment robot_al5d copied to \Users\lboloni\Documents\Code\_ExternalExp\robot_al5d
***ExpRun**: Experiment demonstration copied to \Users\lboloni\Documents\Code\_ExternalExp\demonstration
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\sensorprocessing_propriotuned_cnn\resnet50_128_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: sensorprocessing_propriotuned_cnn/resnet50_128 successfully loaded
***ExpRun**: Exp/run variant sensorprocessing_propriotuned_cnn/boo created in \Users\lboloni\Documents\Code\_ExternalExp\sensorprocessing_propriotuned_cnn\boo.yaml


***ExpRun**: Experiment robot_al5d copied to \Users\lboloni\Documents\Code\_ExternalExp\robot_al5d
***ExpRun**: Experiment demonstration copied to \Users\lboloni\Documents\Code\_ExternalExp\demonstration


In [13]:
# The experiment/run we are going to run: the specified model will be created
experiment = "sensorprocessing_propriotuned_cnn"
# run = "vgg19_128"
# run = "resnet50_128"
# run = "vgg19_256"
# run = "resnet50_256"
run = "boo"
exp = Config().get_experiment(experiment, run)

exp_robot = Config().get_experiment(exp["robot_exp"], exp["robot_run"])

***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\sensorprocessing_propriotuned_cnn\boo_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: sensorprocessing_propriotuned_cnn/boo successfully loaded
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\robot_al5d\position_controller_00_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: robot_al5d/position_controller_00 successfully loaded


### Create regression training data (image to proprioception)
The training data (X, Y) is all the pictures from a demonstration with the corresponding proprioception data. 

In [14]:
def load_images_as_proprioception_training(exp: Experiment, exp_robot: Experiment):
    """Loads the training images specified in the exp/run. Processes them as two tensors as input and target data for proprioception training. 
    Caches the processed results into the input and target file specified in the exp/run. 
    
    Remove those files to recalculate
    """
    retval = {}
    proprioception_input_path = pathlib.Path(exp.data_dir(), "proprio_input.pth")
    proprioception_target_path = pathlib.Path(exp.data_dir(), "proprio_target.pth")

    if proprioception_input_path.exists():
        retval["inputs"] = torch.load(proprioception_input_path, weights_only=True)
        retval["targets"] = torch.load(proprioception_target_path, weights_only=True)
    else:
        inputlist = []
        targetlist = []
        transform = sp_helper.get_transform_to_sp(exp)
        for val in exp["training_data"]:
            run, demo_name, camera = val
            #run = val[0]
            #demo_name = val[1]
            #camera = val[2]
            exp_demo = Config().get_experiment("demonstration", run)
            demo = Demonstration(exp_demo, demo_name)
            for i in range(demo.metadata["maxsteps"]):
                sensor_readings, _ = demo.get_image(i, device=device, transform=transform, camera=camera)
                inputlist.append(sensor_readings[0])
                a = demo.get_action(i)
                rp = RobotPosition.from_vector(exp_robot, a)
                anorm = rp.to_normalized_vector(exp_robot)        
                targetlist.append(torch.from_numpy(anorm))
        retval["inputs"] = torch.stack(inputlist)
        retval["targets"] = torch.stack(targetlist)
        torch.save(retval["inputs"], proprioception_input_path)
        torch.save(retval["targets"], proprioception_target_path)

    # Separate the training and validation data. 
    # We will be shuffling the demonstrations 
    length = retval["inputs"].size(0)
    rows = torch.randperm(length) 
    shuffled_inputs = retval["inputs"][rows]
    shuffled_targets = retval["targets"][rows]

    training_size = int( length * 0.67 )
    retval["inputs_training"] = shuffled_inputs[1:training_size]
    retval["targets_training"] = shuffled_targets[1:training_size]

    retval["inputs_validation"] = shuffled_inputs[training_size:]
    retval["targets_validation"] = shuffled_targets[training_size:]

    return retval

In [15]:
exp["training_data"]

[['freeform', '2024_12_26__16_40_20', 'dev2'],
 ['freeform', '2024_12_26__16_44_06', 'dev2']]

In [16]:
tr = load_images_as_proprioception_training(exp, exp_robot)
inputs_training = tr["inputs_training"]
targets_training = tr["targets_training"]
inputs_validation = tr["inputs_validation"]
targets_validation = tr["targets_validation"]

***ExpRun**: Experiment default config \Users\lboloni\Documents\Code\_ExternalExp\demonstration\_defaults_demonstration.yaml was empty, ok.
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\demonstration\freeform_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: demonstration/freeform successfully loaded
***ExpRun**: Experiment default config \Users\lboloni\Documents\Code\_ExternalExp\demonstration\_defaults_demonstration.yaml was empty, ok.
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\demonstration\freeform_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: demonstration/freeform successfully loaded


### Create a model that performs proprioception regression

In [17]:

if exp['model'] == 'VGG19ProprioTunedRegression':
    model = VGG19ProprioTunedRegression(exp, device)
elif exp['model'] == 'ResNetProprioTunedRegression':
    model = ResNetProprioTunedRegression(exp, device)
else:
    raise Exception(f"Unknown model {exp['model']}")

if exp['loss'] == 'MSELoss':        
    criterion = nn.MSELoss()
elif exp['loss'] == 'L1Loss':
    criterion = nn.L1Loss()

optimizer = optim.Adam(model.parameters(), lr=exp['learning_rate'])

c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
# Create DataLoaders for batching
batch_size = exp['batch_size']
train_dataset = TensorDataset(inputs_training, targets_training)
test_dataset = TensorDataset(inputs_validation, targets_validation)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [19]:
def train_and_save_proprioception_model(model, criterion, optimizer, modelfile, device="cpu", epochs=20):
    """Trains and saves the proprioception model
    FIXME: must have parameters etc to investigate alternative models. 
    """

    model = model.to(device)
    criterion = criterion.to(device)
    # Training loop
    num_epochs = epochs
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch_X, batch_y in train_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            predictions = model.forward(batch_X)
            loss = criterion(predictions, batch_y)            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        if (epoch + 1) % 1 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}')

    # Evaluate the model
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)
            test_loss += loss.item()

    test_loss /= len(test_loader)
    print(f'Test Loss: {test_loss:.4f}')
    torch.save(model.state_dict(), modelfile)

In [20]:
modelfile = pathlib.Path(
    exp["data_dir"], exp["proprioception_mlp_model_file"])
epochs = exp["epochs"]
if modelfile.exists():
    print("*** Train-Propriotuned-CNN ***: NOT training; model already exists, loading it")
    model.load_state_dict(torch.load(modelfile))
else:
    train_and_save_proprioception_model(model, criterion, optimizer, modelfile, device=device, epochs=epochs)

Epoch [1/17], Loss: 5.6413
Epoch [2/17], Loss: 2.2211
Epoch [3/17], Loss: 1.8814
Epoch [4/17], Loss: 1.6540
Epoch [5/17], Loss: 1.5371
Epoch [6/17], Loss: 1.2730
Epoch [7/17], Loss: 1.1718
Epoch [8/17], Loss: 1.1122
Epoch [9/17], Loss: 0.9304
Epoch [10/17], Loss: 1.0891
Epoch [11/17], Loss: 0.8779
Epoch [12/17], Loss: 0.7981
Epoch [13/17], Loss: 0.8194
Epoch [14/17], Loss: 0.8142
Epoch [15/17], Loss: 0.7100
Epoch [16/17], Loss: 0.7902
Epoch [17/17], Loss: 0.8364
Test Loss: 0.9624
